__Lab 05 - SCA (Part 2): Timing Attack__

Student Names: [insert your full name(s) here]

Group Number: [insert your group number as in Canvas here]

# Setup

## Importing Python Libraries & Setup ChipWhisperer

Begin by importing the required libraries, connecting to the scope, and applying the default setup:

Next, compile the firmware:

Program the microcontroller with the "training-CWNANO" hex file:

## Creating Functions to Reset & Read Target

These functions allow you to reset the target from your code and read any text received from the device:

__These functions are completed, no need to change__

In [ ]:
import time
def reset_target(): 
    target.flush()
    scope.io.nrst = 'low'
    time.sleep(0.2)
    scope.io.nrst = 'high'
    time.sleep(0.01)

def readall_target():
    ret = ""
    num_char = target.in_waiting()
    while num_char > 0:
        ret += target.read(timeout=10)
        time.sleep(0.1)
        num_char = target.in_waiting()
    return ret

print("✔️ OK to continue!")

Call those two functions here (print the output of readall_target())

In [ ]:
# Should print start-up test from the device
reset_target()
print(readall_target())
print("✔️ OK to continue!")

# Testing Target w/Example Password

This functions allow you to write (i.e., send) a password to the device and read back the result.

Note: There are 2 different (but similar) versions of the password checker: `password_checker_v1` and `password_checker_v2`. Use "cmd" to select the version of the password checker function: 'a' for v1 and 'b' for v2.

__This function is completed, no need to change__

In [ ]:
# Function: get_pwr_trace()
#   Input:  'password_guess' - input string of max 16 characters (it also accepts shorter strings)
#   Input:  'cmd'            - input character command to select the respective function from the CW
#   Output: 'msg'            - returns "ACCESS GRANTED" if correct password, otherwise "WRONG PASSWORD"
def check_password(password_guess, cmd='a'):
    # reset target and read start-up text
    reset_target()
    # print(readall_target()) # use this to print the start-up text
    readall_target() # use this to not print the start-up text

    # encode password as bytes
    # then append 0s at the end because it expects 16 bytes
    b = bytearray(password_guess.encode())
    b += bytes(16 - len(b))
    
    # send the encoded password
    target.simpleserial_write(cmd, b)
    time.sleep(0.01)
    
    # reads output text from device
    # Note: [:-4] removes the last 4 char added extra by the target, which are useless.
    msg = readall_target()[:-4]
    
    return msg
    
print("✔️ OK to continue!")

Here is an example of resetting the target, writing a password to the device, and reading back the result.

For the training file, the correct password is "USFCSE".

__These code cells are completed, no need to change__

In [ ]:
password = "USFCSE"

msg = check_password(password, 'a')

print(msg)

# do not change these lines below:
assert msg == "ACCESS GRANTED"
print("✔️ OK to continue!")

In [ ]:
password = "BadPassword"

msg = check_password(password, 'a')

print(msg)

# do not change these lines below:
assert msg == "WRONG PASSWORD"
print("✔️ OK to continue!")

# Recording Traces

Now that we can communicate with our super-secure system, our next goal is to get a power trace while the target is running. To do this, we'll arm the scope just before we send our password attempt, then record the trace as we've done before.

In [ ]:
# You may decrease this for faster speed. But IF too small power sample size, 
# then you may not collect the last part of the running firmware on the CW.

scope.adc.samples = 800
print("✔️ OK to continue!")

First we'll make a function to guess a password and return a power trace, since we'll be repeating those steps a lot.

Note: There are 2 different (but similar) versions of the password checker: password_checker_v1 and password_checker_v2. Use "cmd" to select the version of the password checker function: 'a' for v1 and 'b' for v2.

__This function is completed, no need to change__

In [ ]:
# Function: get_pwr_trace()
#   Input:  'password_guess' - input string of max 16 characters (it also accepts shorter strings)
#   Input:  'cmd'            - input character command to select the respective function from the CW
#   Output: 'trace'          - power trace collected from the CW as a Numpy Array
def get_pwr_trace(password_guess, cmd='a'):
    # reset target and read start-up text
    reset_target()
    # print(readall_target()) # use this to print the start-up text
    readall_target() # use this to not print the start-up text

    scope.arm()
    b = bytearray(password_guess.encode())
    b += bytes(16 - len(b))
    target.simpleserial_write(cmd, b)
    
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    trace = scope.get_last_trace()
    return trace

print("✔️ OK to continue!")

Test this function:

__Complete missing parts__

In [ ]:
trace = get_pwr_trace('''what should you write here?''')

print(trace)

# do not change these lines below:
assert type(trace) == np.ndarray
assert len(trace)  > 0
print("✔️ OK to continue!")

# Timing Analysis - Generate & Plot Traces

Now that we can generate traces for any given passwords, then we can try to analyze the power consumption and figure out how to use this information to find the password. But just 1 plot might not be enough (as you can see from previous example). We need to compare an contrast different power examples.

## Test 1: Plot multiple traces for passwords with varying number of correct letters

There are many ways we can attempt to analyze and compare the powers, but we need to start from somewhere.

Since we want to detect the power difference between different passwords, lets collect and plot multiple traces with slight variations: 0, 1, 2, 3, 4, 5, and 6 correct characters. Does this give you any more information on how to attack?

## Test 2: Plot difference between traces of varying number of correct letters

It is hard to distinguish each graph. So, let's try instead plotting the differences between each trace. Does this give you any more information on how to attack?

## Test 3: Plot traces for passwords with same vs different number of correct letters

Try between traces with same number of correct characters (e.g., 3 correct) vs finding an additional correct character. Does this give you any more information on how to attack?

Base on the previous plots, can you figure out any information leak? Can you infer any hypothesis on how to attack the password checkers?

If so, us this information for your attack!

# Attacking a Single Letter

Now that we've located a distinctive timing difference, we can start building our attack. We'll start with a single letter, since that will quickly give us some feedback on the attack.

The plan for the attack is simple: keep guessing letters until we no longer see the distinctive power traces. To do this, we'll create a loop that:

* Figures out our next character guess
* Does the capture and records the trace
* Checks if the new trace significantly differs or not from a previous (reference) trace.

Assume the password only accepts English alphabetical letter (both uppercase and lowercase).

Since we are going to need specify this criteria many time, let's make a list of char called `char_list` that contains all possible  password characters accepted by the system. This will make it easier for following steps:

__Complete missing parts__

In [ ]:
char_list = []

First, to make things a little easier for later, we'll make a function that will compare whether our power traces are (guess incorrect) or aren't (guess correct) very similar.

If they are not similar (guess incorrect), then return the approximate position (sample number) where the difference starts. Otherwise, return "False" (boolean).

__Complete missing parts__

In [ ]:
# Function: comp_traces()
#   Input:  'trace1' - input power trace 1
#   Input:  'trace1' - input power trace 2
#   Output: 'pos'    - If they are not similar (guess incorrect), then return the approximate 
#                      position (sample number) where the difference starts. 
#                      Otherwise, return "False" (boolean).
def comp_traces(trace1: np.ndarray, trace2: np.ndarray):
    return pos

Test your function:

__These code cells are completed, no need to change__

In [ ]:
# Test function 1:
trace_1correct = get_pwr_trace("Uxxxxx", 'a')
trace_2correct = get_pwr_trace("USxxxx", 'a')
assert(comp_traces(trace_1correct, trace_2correct) != False)
print("✔️ OK to continue!")

In [ ]:
# Test function 2:
trace_3correct_v1 = get_pwr_trace("USFxxx", 'a')
trace_3correct_v2 = get_pwr_trace("USFabc", 'a')
assert(comp_traces(trace_3correct_v1, trace_3correct_v2) == False)
print("✔️ OK to continue!")

Now, create a function that will find the next correct character (if any) for a given reference password (ref_passwd). If it found the next correct character, then return:
* the next correct character,
* the respective power trace for the password tested with the next correct password,
* and the approximate position (pos) where this trace first differs the trace of the reference password

Note: There are 2 different (but similar) versions of the password checker: password_checker_v1 and password_checker_v2. Use "cmd" to select the version of the password checker function: 'a' for v1 and 'b' for v2.

__Complete missing parts__

In [ ]:
# Function: comp_traces()
#   Input:  'ref_password' - input reference password to find the next correct character
#   Input:  'cmd'          - input character command to select the respective function from the CW
#   Input:  'char_list'    - input character list of possible allowed char in the password
#   Output: 'ch'           - next correct character. 
#   Output: 'trace_test'   - the respective power trace for the password tested with the next correct password. 
#   Output: 'pos'          - approximate position (sample number) where the difference starts. 
def find_letter(ref_password: str, cmd: str, char_list=char_list):
    return ch, test_trace, pos

## Test code with training hex file: "training-CWNANO.hex"

Since this is not the full password attack, then just test with cmd='a'

We are giving you 4 test functions for you to try:

__These code cells are completed, no need to change__

In [ ]:
# Test function 1:
ch, trace, pos = find_letter("", 'a', char_list)
print(f"ch = {ch}; pos = {pos}")
assert(ch == 'U')
print("✔️ OK to continue!")

In [ ]:
# Test function 2:
ch, trace, pos = find_letter("U", 'a', char_list)
print(f"ch = {ch}; pos = {pos}")
assert(ch == 'S')
print("✔️ OK to continue!")

In [ ]:
# Test function 3:
ch, trace, pos = find_letter("USF", 'a', char_list)
print(f"ch = {ch}; pos = {pos}")
assert(ch == 'C')
print("✔️ OK to continue!")

In [ ]:
# Test function 4:
ch, trace, pos = find_letter("USFCSE", 'a', char_list)
print(f"ch = {ch}; pos = {pos}")
assert(ch == False)
print("✔️ OK to continue!")

# Attacking the Full Password

Now that we can guess a single character, attacking the rest is easy. We just need to repeat the process in another loop, and update our password guess (passwd_guess) with the new correct char. You should start with an empty string as your initial reference password to then find the first correct character.

For each time you find a new correct character:
* print this correct character found,
* save the respective trace in a list (trace_list),
* and save the respective approximate positions where it first differs in another list (pos_list)

Once you found all the correct characters, then return
* the complete "passwd_guess",
* the "trace_list",
* and the "pos_list"

After updating the below script and running it, you should see parts of the password printed out as each letter is found.

Note: There are 2 different (but similar) versions of the password checker: password_checker_v1 and password_checker_v2. Use "cmd" to select the version of the password checker function: 'a' for v1 and 'b' for v2.

__Complete missing parts__

In [ ]:
# Function: comp_traces()
#   Input:  'cmd'            - input character command to select the respective function from the CW
#   Input:  'char_list'      - input character list of possible allowed char in the password
#   Output: 'password_guess' - final password guess with all the correct characters 
#   Output: 'trace_list'     - list of power traces for each correct substring password
#   Output: 'pos_lits'       - list of approximate position (sample number) where the difference starts  
def find_password(cmd: str, char_list=char_list):
    return password_guess, trace_list, pos_list

## Test code with training hex file: "training-CWNANO.hex"

### Test with cmd='a'

Find the password with `password_checker_v1`

__Complete missing parts__

In [ ]:
# "training-CWNANO.hex"; cmd='a'
password, trace_list, pos_list = find_password('''what should go here?''')

print("Password:", password)

# Verify password
msg = check_password('''what should go here?''')
print(f"Confirm {password}: {msg}")

# do not change these lines below:
assert msg == "ACCESS GRANTED"
print("✔️ OK to continue!")

Plot all the traces stored in "trace_list" and annotate where each trace starts to differ or with significant peaks (use "pos_list")

### Test with cmd='b'

Find the password with `password_checker_v2`

__Complete missing parts__

In [ ]:
# "training-CWNANO.hex"; cmd='b'
password, trace_list, pos_list = find_password('''what should go here?''')

print("Password:", password)

# Verify password
msg = check_password('''what should go here?''')
print(f"Confirm {password}: {msg}")

# do not change these lines below:
assert msg == "ACCESS GRANTED"
print("✔️ OK to continue!")

Plot all the traces stored in "trace_list" and annotate where each trace starts to differ or with significant peaks (use "pos_list")

# Attacking a Secret Password

After you verified your code correct functionality with the training file, now re-use the same code to find the secret password in the "attack-CWNANO.hex" file. Notice that we have not given you the ".c" file, so you do not know the password.

Note: "attack-CWNANO.hex" is the same code as "training-CWNANO.hex" but with a different secret password that you need to find. If you implemented previous steps and functions correctly, you should not have any issues 

Run this code to program the ChipWhisperer's microcontroller with the attack hex file:

__This code cell is completed, no need to change__

In [ ]:
prog = cw.programmers.STM32FProgrammer
fw_path = '../Firmware/attack-CWNANO.hex'
cw.program_target(scope, prog, fw_path)

## Test code with arrack hex file: "attack-CWNANO.hex"

### Test with cmd='a'

Find the password with `password_checker_v1`

__Complete missing parts__

In [ ]:
# "attack-CWNANO.hex"; cmd='a'
password, trace_list, pos_list = find_password('''what should go here?''')

print("Password:", password)

# Verify password
msg = check_password('''what should go here?''')
print(f"Confirm {password}: {msg}")

# do not change these lines below:
assert msg == "ACCESS GRANTED"
print("✔️ OK to continue!")

Plot all the traces stored in "trace_list" and annotate where each trace starts to differ or with significant peaks (use "pos_list")

### Test with cmd='b'

Find the password with `password_checker_v2`

__Complete missing parts__

In [ ]:
# "attack-CWNANO.hex"; cmd='b'
password, trace_list, pos_list = find_password('''what should go here?''')

print("Password:", password)

# Verify password
msg = check_password('''what should go here?''')
print(f"Confirm {password}: {msg}")

# do not change these lines below:
assert msg == "ACCESS GRANTED"
print("✔️ OK to continue!")

Plot all the traces stored in "trace_list" and annotate where each trace starts to differ or with significant peaks (use "pos_list")

# Attempting to Attack Your Countermeasure

After you successfully attacked the password checkers in "attack-CWNANO.hex" file and identified the secret password, then now apply some countermeasures.

* Resuse the same "training.c" file. Open it and make a copy of either function: `password_checker_v1` or `password_checker_v2` (whichever you prefer). Rename the new function to `password_checker_v3`.
    
* Modify the `password_checker_v3` with a countermeasure that prevents your full password attack.

* Add this new function callback in the `main` method, using simpleserial parameters `'c'`.

* Retry the same full password attack on this improved password checker. It should thwart your attack in previous parts. Save outputs showing the waveforms you have identified as belonging to the countermeasure password checking routine.

Use the same `"USFCSE"` password as the other password checkers in "training.c"

First, you need to recomplie "training.c" as you did in section 1.1:

The, re-program the microcontroller with the new generated "training-CWNANO" hex file as you did in section 1.1"

## Verify new password checker with countermeasure

First, verify correctness of your `password_checker_v3`. Pass the correct password "USFCSE" to see if it grants access as it should rather than by default always deny access.

__Complete missing parts__

In [ ]:
# Pass the correct password (hardcoded) for cmd='c'
msg = check_password('''what should go here?''')
print(f"Confirm USFCSE: {msg}")

# do not change these lines below:
assert msg == "ACCESS GRANTED"
print("✔️ OK to continue!")

Now, pass an incorrect password to see if it denies access as it should rather than by default always grant access.

__Complete missing parts__

In [ ]:
# Pass an incorrect password (hardcoded) for cmd='c'
msg = check_password('''what should go here?''')
print(f"Confirm USFCSE: {msg}")

# do not change these lines below:
assert msg == "WRONG PASSWORD"
print("✔️ OK to continue!")

## Attempt attack on countermeasure

Attempt to attack `password_checker_v3` with your same full password attack.

__Complete missing parts__

In [ ]:
# "training-CWNANO.hex"; cmd='c'
password, trace_list, pos_list = find_password('''what should go here?''')

print("Password:", password)

# Verify password
msg = check_password('''what should go here?''')
print(f"Confirm {password}: {msg}")

# do not change these lines below:
assert msg == "WRONG PASSWORD"
print("✔️ OK to continue!")

Plot all the traces stored in "trace_list" and annotate where each trace starts to differ or with significant peaks (use "pos_list")

# Diconnect Scope/Target

Disconnect the CW's scope and target:

In [ ]:
scope.dis()
target.dis()